# Stochastic regression

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pandas as pd
import scipy
from scipy.stats import chi2

In [2]:
# define reward function
def get_noise():
    return  np.random.binomial(1,0.5,1)[0] - 0.5

def get_unitv():
    # generate unit vector in sphere
    x = np.random.normal(0,1,2)
    return x/np.linalg.norm(x)

# stochastic regression
def stoc(n):
    # y = 0.3 + 0.3 x + eps
    x = np.zeros(n)
    eps = np.random.normal(0,1,n)
    # x_{n+1} = mean(x_{1:n}) + np.mean(eps_{1:n})
    for i in range(n):
        if i>0:
            c = 1
            x[i] = np.mean(x[:i]) + c*np.mean(eps[:i])
    y = 0.3 + x*0.3 + eps 
    
    X = np.concatenate((np.ones((n,1)), x.reshape(-1,1)), axis = 1)
    return  X,y

## $\Sigma_0 = e^2 \log(n)$

In [3]:
def Volumn(V):
    # given a matrix V, compute the volumn correspoing to the ellipse of the format x^T V x \leq 1
    # universal constant are dropped
    s =  np.linalg.svd(V)[1]
    return 1/np.product(np.sqrt(s))

def scaled_estimators(n, lam, sampler):
    
    # consider confidence level 0.8, 0.85, 0.9
    cql = chi2.ppf([0.8, 0.85, 0.9], 2)
    true_theta = np.array([0.3, 0.3]).reshape(-1,1)
    CI_data = np.zeros((2,15))
    
    X,y = sampler(n)
    #######################################
    # compute OLS
    M = LinearRegression(fit_intercept = False).fit(X, y)
    coeff = M.coef_.copy().reshape(-1,1)
    Sn = X.T@X
    residual = y.reshape(-1,1) - X@coeff
    # compute residual and use it as an estimate of sigma^2
    sig_h = np.sqrt(np.mean(residual**2))
    
    c = 1/sig_h**2*(coeff.reshape(1,-1)- true_theta.reshape(1,-1))@Sn@(coeff - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = Volumn(Sn)*r
        v = np.log(v)
        CI_data[:,0 + 5*i] = np.array([cover, v]).reshape(-1)
    
    #######################################
    # W-decorrelation
    W_lambdas = np.ones(n)*lam
    # Latest parameter estimate vector
    wols = M.coef_.copy()
    # Latest w_t vector
    w = np.zeros((2,1))
    # Latest matrix W_tX_t = w_1 x_1^T + ... + w_t x_t^T
    WX = np.zeros((2,2))
    # Latest vector of marginal variances reward_vars * (w_1**2 + ... + w_t**2)
    W = np.zeros_like(X)
    for t in range(n):
        # Update w_t = (1/(norm{x_t}^2+lambda_t)) (x_t - W_{t-1} X_{t-1} x_t)
        w = (np.eye(2) - WX)@X[t,:].reshape(-1,1) 
        w = w/(np.sum(X[t,:]**2) + W_lambdas[t])
        W[t, :] = w.reshape(-1)
        # Update beta_t = beta_{t-1} + w_t (y_t - <beta_OLS, x_t>)
        wols += w.reshape(-1) * (y[t] - np.sum(X[t,:]*coeff.reshape(-1)) ) ###
        # Update W_tX_t = W_{t-1}X_{t-1} + w_t x_t^T 
        WX = WX + w@X[t,:].reshape(1,-1)
        
    WW = np.linalg.inv(W.T@W)
    r = cq**1.5*sig_h**3
   
    # compute statistic
    c = 1/sig_h**2*(wols - true_theta.reshape(1,-1))@WW@(wols.reshape(-1,1) - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = Volumn(WW)*r
        v = np.log(v)
        CI_data[:,1 + 5*i] = np.array([cover, v]).reshape(-1)
    
    
    
    ######################################
    #  Modified ALEE - log(n)
    Z = np.zeros_like(X)
    for t in range(n):
        if t == 0:
            S = np.log(n)*np.eye(2)
        elif t == 1:
            Xsub = X[:t,:].reshape(-1,2)
            S = np.log(n)*np.eye(2) + Xsub.T@Xsub
        else:
            Xsub = X[:t,:]
            S = np.log(n)*np.eye(2) + Xsub.T@Xsub
        
        S_invh = scipy.linalg.sqrtm(np.linalg.inv(S))
        
        zt = S_invh@X[t,:].reshape(-1,1)
        Z[t,:] = zt.reshape(-1)
    
    
    W = np.zeros_like(Z)
    for i in range(n):
        if i==0:
            z = Z[0,:].reshape(1,-1)
            vtt = np.eye(2)
            vt = np.linalg.inv(np.eye(2) + z.T@z)
        elif i == 1:
            z = Z[:(i+1),:]
            vtt = vt
            vt = np.linalg.inv(np.eye(2) + z.T@z)
        else:
            z = Z[:(i+1),:]
            vt = np.linalg.inv(np.eye(2) + z.T@z)
            z = Z[:i,:]
            vtt = np.linalg.inv(np.eye(2) + z.T@z)
        
        w = vt@Z[i,:].reshape(-1,1)
        
        w_factor = np.sqrt(1 + Z[i,:].reshape(1,-1)@vtt@Z[i,:].reshape(-1,1))
        w = w_factor.reshape(-1)[0]*w
        
        W[i,:] = w.reshape(-1)
        
    
    ######### design new Zt
    kappa = np.log(n)
    tau = np.exp(1)*np.sqrt(np.log(n))
    Z1 = np.copy(Z)
    Vn_inv = np.eye(2) + Z1.T@Z1
    evalues, evectors = np.linalg.eig(Vn_inv)
    
    addn = 0
    # construct new Z
    for i in range(2):
        if evalues[i] < kappa:
            dn = int((kappa - evalues[i])*tau)
            addn = addn + dn
            Z1 = np.concatenate((Z1,np.tile(evectors[:,i], (dn,1))/tau), axis = 0)
    
    
    addW = np.zeros((addn,2))
    
    for i in range(addn):
    
        z = Z1[:(i+n+1),:]
        vt = np.linalg.inv(np.eye(2) + z.T@z)
        z = Z1[:(i+n),:]
        vtt = np.linalg.inv(np.eye(2) + z.T@z)
        w = vt@Z1[i,:].reshape(-1,1)
        w_factor = np.sqrt(1 + Z1[i,:].reshape(1,-1)@vtt@Z1[i,:].reshape(-1,1))
        w = w_factor.reshape(-1)[0]*w
        
        addW[i,:] = w.reshape(-1)
    
    # simulate gaussian random variable
    new_error = np.random.normal(0, sig_h, addn).reshape(-1,1)
    
    # design new estimator
    A = W.T@X
    alee = np.linalg.inv(A)@(W.T@y.reshape(-1,1) + addW.T@new_error)
    
    OSn = A.T@A
    c = 1/sig_h**2*(alee.reshape(1,-1)- true_theta.reshape(1,-1))@OSn@(alee - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = Volumn(OSn)*r
        v = np.log(v)
        CI_data[:,2 + 5*i] = np.array([cover, v]).reshape(-1)

    
    
    ######################################
    # ALEE - n**0.5
    
    Z = np.zeros_like(X)
    for t in range(n):
        if t == 0:
            S = np.exp(2)*np.eye(2)*n**0.5
        elif t == 1:
            Xsub = X[:t,:].reshape(-1,2)
            S = np.exp(2)*np.eye(2)*n**0.5 + Xsub.T@Xsub
        else:
            Xsub = X[:t,:]
            S = np.exp(2)*np.eye(2)*n**0.5 + Xsub.T@Xsub
        
        S_invh = scipy.linalg.sqrtm(np.linalg.inv(S))
        
        zt = S_invh@X[t,:].reshape(-1,1)
        Z[t,:] = zt.reshape(-1)
    
    
    W = np.zeros_like(Z)
    for i in range(n):
        if i==0:
            z = Z[0,:].reshape(1,-1)
            vtt = np.eye(2)
            vt = np.linalg.inv(np.eye(2) + z.T@z)
        elif i == 1:
            z = Z[:(i+1),:]
            vtt = vt
            vt = np.linalg.inv(np.eye(2) + z.T@z)
        else:
            z = Z[:(i+1),:]
            vt = np.linalg.inv(np.eye(2) + z.T@z)
            z = Z[:i,:]
            vtt = np.linalg.inv(np.eye(2) + z.T@z)
        
        w = vt@Z[i,:].reshape(-1,1)
        
        w_factor = np.sqrt(1 + Z[i,:].reshape(1,-1)@vtt@Z[i,:].reshape(-1,1))
        w = w_factor.reshape(-1)[0]*w
        
        W[i,:] = w.reshape(-1)
        
    
    ######### design new Zt
    kappa = np.log(n)
    tau = np.exp(1)*n**0.25
    Z1 = np.copy(Z)
    Vn_inv = np.eye(2) + Z1.T@Z1
    evalues, evectors = np.linalg.eig(Vn_inv)
    
    addn = 0
    # construct new Z
    for i in range(2):
        if evalues[i] < kappa:
            dn = int((kappa - evalues[i])*tau)
            addn = addn + dn
            Z1 = np.concatenate((Z1,np.tile(evectors[:,i], (dn,1))/tau), axis = 0)
    
    
    addW = np.zeros((addn,2))
    
    for i in range(addn):
    
        z = Z1[:(i+n+1),:]
        vt = np.linalg.inv(np.eye(2) + z.T@z)
        z = Z1[:(i+n),:]
        vtt = np.linalg.inv(np.eye(2) + z.T@z)
        w = vt@Z1[i,:].reshape(-1,1)
        w_factor = np.sqrt(1 + Z1[i,:].reshape(1,-1)@vtt@Z1[i,:].reshape(-1,1))
        w = w_factor.reshape(-1)[0]*w
        
        addW[i,:] = w.reshape(-1)
    
    # simulate gaussian random variable
    new_error = np.random.normal(0, sig_h, addn).reshape(-1,1)
    
    # design new estimator
    A = W.T@X
    alee = np.linalg.inv(A)@(W.T@y.reshape(-1,1) + addW.T@new_error)
    
    OSn = A.T@A
    c = 1/sig_h**2*(alee.reshape(1,-1)- true_theta.reshape(1,-1))@OSn@(alee - true_theta)
    c = c.reshape(-1)[0]
    for i in range(3):
        cq = cql[i]
        cover = c < cq - 0
        r = cq**1.5*sig_h**3
        v = Volumn(OSn)*r
        v = np.log(v)
        CI_data[:,3 + 5*i] = np.array([cover, v]).reshape(-1)

    
    
    
    
    ######################################
    # ridge concentration
    
    rols = np.linalg.inv(X.T@X + np.eye(2))@X.T@y.reshape(-1,1)
    Vt = np.eye(2) + Sn
    delta_list = [0.2, 0.15, 0.1]
    
    c = (rols.reshape(1,-1) - true_theta.reshape(1,-1))@Vt@(rols - true_theta)
    c = c.reshape(-1)[0]
    
    for i in range(3):
        delta = delta_list[i]
        cq = sig_h*np.sqrt(np.log(np.linalg.det(Vt)/delta**2)) + 1
        cover = c < cq**2 - 0
        v = Volumn(Vt)*cq**3
        v = np.log(v)
        CI_data[:,4 + 5*i] = np.array([cover, v]).reshape(-1)
    

    return CI_data
    
    
    
#   replication function
def ACV_repli(N , n, lam,  sampler):
    E1 = np.zeros((N, 10))
    E2 = np.zeros((N, 10))
    E3 = np.zeros((N, 10))
    
    for i in range(N):
        output = scaled_estimators(n, lam, sampler)
        E1[i,:] =  output[:,:5].reshape(1,10)
        E2[i,:] =  output[:,5:10].reshape(1,10)
        E3[i,:] =  output[:,10:15].reshape(1,10)
        
    return (E1,E2,E3)

In [4]:
def compute_lam(N, n, sampler):
    
    record = np.zeros(N)
    for k in range(N):
        
        X,y = sampler(n)

        evalue, evector = np.linalg.eig(X.T@X)
        
        record[k] = np.min(evalue)
        
    return np.percentile(record, 10)

In [5]:
np.random.seed(517)
lam = compute_lam(1000, 1000, stoc)/np.log(1000)

In [6]:
np.random.seed(517)
E1,E2,E3 = ACV_repli(1000, 1000, lam, stoc)


np.save('E1', E1)
np.save('E2', E2)
np.save('E3', E3)

# Confidence level 0.8

## First row represents coverage ; second row represents average log (Volume)

In [7]:
df = pd.DataFrame(np.mean(E1, axis = 0).reshape(2,5), columns = ['ols','W', 'ALEE-logn','ALEE-n', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,ALEE-n,Concentration
0,0.000,0.185,0.930,0.964,1.000
1,-2.555,0.812,-0.876,0.116,0.017


## Standard errors for empirical coverage and logarithm of volume

In [8]:
df = pd.DataFrame(np.std(E1, axis = 0).reshape(2,5), columns = ['ols','W', 'ALEE-logn','ALEE-n', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,ALEE-n,Concentration
0,0.00,0.388,0.255,0.186,0.000
1,0.32,0.167,0.431,0.501,0.244


# Confidence level 0.85

In [9]:
df = pd.DataFrame(np.mean(E2, axis = 0).reshape(2,5), columns = ['ols','W', 'ALEE-logn','ALEE-n', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,ALEE-n,Concentration
0,0.000,0.222,0.95,0.972,1.000
1,-2.308,1.059,-0.63,0.363,0.071


## Standard errors for empirical coverage and logarithm of volume

In [10]:
df = pd.DataFrame(np.std(E2, axis = 0).reshape(2,5), columns = ['ols','W', 'ALEE-logn','ALEE-n', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,ALEE-n,Concentration
0,0.00,0.416,0.218,0.165,0.000
1,0.32,0.167,0.431,0.501,0.246


# Confidence level 0.9

In [11]:
df = pd.DataFrame(np.mean(E3, axis = 0).reshape(2,5), columns = ['ols','W', 'ALEE-logn','ALEE-n', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,ALEE-n,Concentration
0,0.001,0.279,0.974,0.989,1.000
1,-2.017,1.350,-0.339,0.653,0.144


## Standard errors for empirical coverage and logarithm of volume

In [12]:
df = pd.DataFrame(np.std(E3, axis = 0).reshape(2,5), columns = ['ols','W', 'ALEE-logn','ALEE-n', 'Concentration'])
round(df,3)

,ols,W,ALEE-logn,ALEE-n,Concentration
0,0.032,0.449,0.159,0.104,0.000
1,0.320,0.167,0.431,0.501,0.249
